In [1]:
import numpy as np
import pydot
from IPython.display import HTML, SVG, display
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder,
                         FindResourceOrThrow, GenerateHtml,
                         InverseDynamicsController, MeshcatVisualizer,
                         MeshcatVisualizerParams, MultibodyPlant, Parser,
                         Simulator, StartMeshcat)

from manipulation import running_as_notebook

In [2]:
filename = "drake/manipulation/models/iiwa_description/sdf/iiwa14_no_collision.sdf"
base_link_frame_name = "iiwa_link_0"
step = 1e-4
position_vector = [0.1, 0, 0.2, 0.2, 1, 0, 0.1]

In [3]:
#start the visualizer
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [4]:
class SimSetup:
    def __init__(self, step, filename, base_link_frame_name, meshcat):
        self.step_size = step
        self.base_link_name = base_link_frame_name
        self.meshcat = meshcat
    def create_builder(self):
        #create a diagram builder object
        builder = DiagramBuilder()
        return builder
    def create_plant_scene_graph(self,builder):
        #plant and scene graph pointers for the newly created multibody plant and scene graph returned
        #they are added to the diagram
        #Makes a new MultibodyPlant with discrete update period time_step and adds it to a diagram builder together with the provided SceneGraph instance, connecting the geometry ports.
        plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=self.step_size)    
        return plant, scene_graph
    
    def create_model(self,plant, scene_graph, filename):
        model = Parser(plant, scene_graph).AddModelFromFile(FindResourceOrThrow(filename))
        plant.WeldFrames(plant.world_frame(), plant.GetFrameByName(self.base_link_name))
        plant.Finalize()
        return model
    
    def add_meshcat2builder(self,builder, scene_graph, meshcat):
        visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
        return visualizer
    
    def setup(self):
        builder = self.create_builder()
        plant, scene_graph = self.create_plant_scene_graph(builder)
        model = self.create_model(plant, scene_graph,  filename)
        visualizer = self.add_meshcat2builder(builder, scene_graph,self.meshcat)
        
        return builder, plant, scene_graph, model, visualizer
    
    
class iiwaRobot:
    def __init__(self,builder, plant, scene_graph, model, visualizer):
        self.builder = builder
        self.plant = plant
        self.scene_graph = scene_graph
        self.model = model
        self.visualizer = visualizer
    
    def iiwa_controller(self):
        Kp = np.full(7, 100)
        Ki = 2 * np.sqrt(Kp)
        Kd = np.full(7, 1)
        iiwa_controller = self.builder.AddSystem(InverseDynamicsController(self.plant, Kp, Ki, Kd, False))
        iiwa_controller.set_name("iiwa_controller");
        #create feedback loop
        #plant state output to controller input
        #controller output to plant actuation input
        self.builder.Connect(self.plant.get_state_output_port(self.model),
                        iiwa_controller.get_input_port_estimated_state())
        self.builder.Connect(iiwa_controller.get_output_port_control(),
                        self.plant.get_actuation_input_port())

        return iiwa_controller
    
    def iiwa_create_diagram(self):
        #create a diagram out of the wired systems
        diagram = self.builder.Build() 
        return diagram
    
    def iiwa_position_set(self, diagram, iiwa_controller, position_vector):    
        #extract context from the diagram
        context = diagram.CreateDefaultContext()
        #extract plant context from the full context
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        q0 = q0 = np.array(position_vector)
        x0 = np.hstack((q0, 0*q0))
        plant.SetPositions(plant_context, q0)
        iiwa_controller.GetInputPort('desired_state').FixValue(iiwa_controller.GetMyMutableContextFromRoot(context), x0)
        return context
    
    def iiwa_animate(self, diagram, context):
        simulator = Simulator(diagram, context)
        simulator.set_target_realtime_rate(1.0)
        self.visualizer.StartRecording()
        simulator.AdvanceTo(5)
        self.visualizer.StopRecording()
        self.visualizer.PublishRecording()

    

In [5]:
simob = SimSetup(step, filename, base_link_frame_name, meshcat)
builder, plant, scene_graph, model, visualizer = simob.setup()

In [6]:
iiwa = iiwaRobot(builder, plant, scene_graph, model, visualizer)
iiwa_controller = iiwa.iiwa_controller()
diagram = iiwa.iiwa_create_diagram()
context = iiwa.iiwa_position_set(diagram, iiwa_controller, position_vector)
iiwa.iiwa_animate(diagram,context)

In [7]:
position_vector = [1, 0, 1, 1.3, 1.5, 0.5, 0.3]
context = iiwa.iiwa_position_set(diagram, iiwa_controller, position_vector)
iiwa.iiwa_animate(diagram,context)